<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [4]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [5]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [7]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag


from oppdeling_trening_testsett import enzymfordeling

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [13]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [14]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28976 entries, 2022-10-31 17:37:00 to 2023-06-14 01:02:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  28976 non-null  int64  
 1   EnzymeType_A2  28976 non-null  int64  
 2   EnzymeType_B   28976 non-null  int64  
 3   EnzymeType_C   28976 non-null  int64  
 4   EnzymeType_D   28976 non-null  int64  
 5   EnzymeType_E   28976 non-null  int64  
 6   RawMatPercent  28976 non-null  float64
 7   NIRfat         28976 non-null  float64
 8   NIRash         28976 non-null  float64
 9   NIRwater       28976 non-null  float64
 10  TT08           28976 non-null  float64
 11  TT20           28976 non-null  float64
 12  TT12           28976 non-null  float64
 13  Mw             455 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [16]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10466 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10466 non-null  int64  
 1   EnzymeType_A2  10466 non-null  int64  
 2   EnzymeType_B   10466 non-null  int64  
 3   EnzymeType_C   10466 non-null  int64  
 4   EnzymeType_D   10466 non-null  int64  
 5   EnzymeType_E   10466 non-null  int64  
 6   RawMatPercent  10466 non-null  float64
 7   NIRfat         10466 non-null  float64
 8   NIRash         10466 non-null  float64
 9   NIRwater       10466 non-null  float64
 10  TT08           10466 non-null  float64
 11  TT20           10466 non-null  float64
 12  TT12           10466 non-null  float64
 13  Mw             332 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [18]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering, trening
# validering og testing

dict_res_1_hel_test = {"modell": [],
                       "test_andel": [],
                       "trening_andel": [],
                       "trening_markert_antall": [],
                       "tid_trening": [],
                       "parametere": [],
                       "RMSE_test": [],
                       "R2_test":[],
                       "MAE_test":[],
                       "MAPE_test":[],
                       "n_runder": [],
                       "n_splitt": [],
                       "tid_optimalisering": []}

dict_res_1_design_test = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_hel_test = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_design_test = copy.deepcopy(dict_res_1_hel_test)

# Uten NIR
dict_res_1_hel_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_1_design_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_hel_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_design_test_NIR = copy.deepcopy(dict_res_1_hel_test)


dict_res_1_hel_trening = {"modell": [],
                          "test_andel": [],
                          "trening_andel": [],
                          "parametere": [],
                          "n_splitt": [],
                          "RMSE_val_snitt": [],
                          "RMSE_val_std": [],
                          "R2_val_snitt":[],
                          "R2_val_std":[],
                          "MAE_val_snitt":[],
                          "MAE_val_std":[],
                          "MAPE_val_snitt":[],
                          "MAPE_val_std":[],
                          "RMSE_trening_snitt": [],
                          "RMSE_trening_std": [],
                          "R2_trening_snitt":[],
                          "R2_trening_std":[],
                          "MAE_trening_snitt":[],
                          "MAE_trening_std":[],
                          "MAPE_trening_snitt":[],
                          "MAPE_trening_std":[],
                          }

dict_res_1_design_trening = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_hel_trening = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_design_trening = copy.deepcopy(dict_res_1_hel_trening)

# Uten NIR
dict_res_1_hel_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_1_design_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_hel_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_design_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Hele treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, \
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                             trening_dict_res=dict_res_1_hel_trening,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50


# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                              trening_dict_res=dict_res_1_hel_trening,
                              treningssett_markert=treningssett_markert,
                              enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                              treningssett_umarkert=treningssett_umarkert,
                              enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                              treningssett=treningssett,
                              testsett=testsett,
                              splitt=splitt,
                              gjentagelser=gjentagelser,
                              n_runder=n_runder,
                              test_andel=test_andel,
                              trenings_andel=trenings_andel,
                              random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        31  34.07
E         15  16.48
B         14  15.38
C         14  15.38
D         13  14.29
A1         4   4.40


## Alternativ 2: Hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Parametervalg for kryssvalidering
splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                            trening_dict_res=dict_res_2_hel_trening,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=trening_umarkert,
                            treningssett=trening,
                            testsett=testsett,
                            splitt=splitt,
                            n_runder=n_runder,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                                trening_dict_res=dict_res_2_hel_trening,
                                rå_data=rå_data,
                                treningssett_markert=treningssett_markert,
                                treningssett_umarkert=treningssett_umarkert,
                                treningssett=treningssett,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        25  27.78
B         16  17.78
C         15  16.67
E         15  16.67
D         13  14.44
A1         6   6.67


## Designproduksjon

## Alternativ 1: Hele treningssettet

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10


# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                             trening_dict_res=dict_res_1_design_trening,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                              trening_dict_res=dict_res_1_design_trening,
                              treningssett_markert=treningssett_markert,
                              enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                              treningssett_umarkert=treningssett_umarkert,
                              enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                              treningssett=treningssett,
                              testsett=testsett,
                              splitt=splitt,
                              gjentagelser=gjentagelser,
                              n_runder=n_runder,
                              test_andel=test_andel,
                              trenings_andel=trenings_andel,
                              random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.39
B         13  19.40
C         13  19.40
D         12  17.91
A2        10  14.93
A1         4   5.97


### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  # Lagring av informasjon om paramtere for treningsandel
  dict_res(dict_res_1_design, 'RFR', test_andel,
           trenings_andel, parametere, rfr_resultater)

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-04-01 16:27:52,925] A new study created in memory with name: no-name-96ec4a62-609a-49bf-aba6-4c0219c7e745


Enzymtyper Fordeling: Treningssett markert andel: 0.4
    Frekvens  Andel
E         24  22.64
B         21  19.81
C         21  19.81
D         19  17.92
A2        15  14.15
A1         6   5.66


[I 2024-04-01 16:28:13,698] Trial 0 finished with value: 734.4439171664216 and parameters: {'n_estimators': 198, 'max_depth': 6}. Best is trial 0 with value: 734.4439171664216.
[I 2024-04-01 16:28:14,034] A new study created in memory with name: no-name-7f595372-25b4-46f0-9e7b-2d93f6e703d2


Enzymtyper Fordeling: Treningssett markert andel: 0.6
    Frekvens  Andel
E         36  22.64
C         32  20.13
B         31  19.50
D         29  18.24
A2        23  14.47
A1         8   5.03


[I 2024-04-01 16:28:31,950] Trial 0 finished with value: 675.4344073050806 and parameters: {'n_estimators': 169, 'max_depth': 7}. Best is trial 0 with value: 675.4344073050806.
[I 2024-04-01 16:28:32,277] A new study created in memory with name: no-name-e703b7e6-eef7-4c2d-97b3-af8adc2b33c0


Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
E         48  22.64
C         43  20.28
B         42  19.81
D         38  17.92
A2        30  14.15
A1        11   5.19


[I 2024-04-01 16:28:46,720] Trial 0 finished with value: 761.993157058165 and parameters: {'n_estimators': 139, 'max_depth': 4}. Best is trial 0 with value: 761.993157058165.
[I 2024-04-01 16:28:46,973] A new study created in memory with name: no-name-0457b5eb-3ac7-43cc-949e-1a75fc3140c0


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
E         60  22.64
C         53  20.00
B         52  19.62
D         48  18.11
A2        38  14.34
A1        14   5.28


[I 2024-04-01 16:29:05,692] Trial 0 finished with value: 553.7133909787998 and parameters: {'n_estimators': 139, 'max_depth': 9}. Best is trial 0 with value: 553.7133909787998.


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.4,771.655579,0.882570,583.790380,0.079633,343.777478,0.976464,257.751708,0.037658
1,0.6,596.060863,0.929933,444.277693,0.056874,311.882719,0.981435,219.741371,0.030999
2,0.8,776.672672,0.881038,608.212310,0.084684,591.927351,0.933709,428.840622,0.062504
3,1.0,528.688348,0.944877,384.145997,0.048472,232.174899,0.989627,154.996714,0.021093


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  # Lagring av informasjon om paramtere for treningsandel
  dict_res(dict_res_1_design, 'KNR', test_andel,
           trenings_andel, parametere, knr_resultater)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-04-01 16:29:06,258] A new study created in memory with name: no-name-b2eb20e8-7d90-4946-a5d2-d0ad61cc315b
[I 2024-04-01 16:29:06,851] Trial 0 finished with value: 928.842141651841 and parameters: {'n_neighbors': 10, 'p': 3}. Best is trial 0 with value: 928.842141651841.
[I 2024-04-01 16:29:06,909] A new study created in memory with name: no-name-e14362f1-7c97-4999-8b9d-1eb326a6ac0d
[I 2024-04-01 16:29:08,363] Trial 0 finished with value: 750.1137400398442 and parameters: {'n_neighbors': 9, 'p': 2}. Best is trial 0 with value: 750.1137400398442.
[I 2024-04-01 16:29:08,450] A new study created in memory with name: no-name-e51cbefa-08bd-4bb2-82fe-7746c927c343
[I 2024-04-01 16:29:09,902] Trial 0 finished with value: 608.5037662690844 and parameters: {'n_neighbors': 6, 'p': 8}. Best is trial 0 with value: 608.5037662690844.
[I 2024-04-01 16:29:09,947] A new study created in memory with name: no-name-32c27be5-215a-4e1f-9b98-f3e66a40d4d3
[I 2024-04-01 16:29:10,725] Trial 0 finished wi

,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.4,849.346109,0.857734,666.850474,0.094371,812.881718,0.868405,607.529581,0.085629
1,0.6,595.361506,0.930097,441.152916,0.058682,577.087850,0.936437,438.588074,0.059593
2,0.8,509.085195,0.948889,400.976825,0.053602,481.677780,0.956103,342.201905,0.046587
3,1.0,551.251267,0.940072,405.863809,0.051577,272.525033,0.985708,183.847936,0.024161


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  # Lagring av informasjon om paramtere for treningsandel
  dict_res(dict_res_1_design, 'SVR', test_andel,
           trenings_andel, parametere, svr_resultater)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-04-01 16:29:10,882] A new study created in memory with name: no-name-c0d8a35e-bc91-460d-8f1d-7a3c4938e248
[I 2024-04-01 16:29:11,578] Trial 0 finished with value: 605.0249778470508 and parameters: {'kernel': 'linear', 'C': 92, 'degree': 15, 'gamma': 'auto'}. Best is trial 0 with value: 605.0249778470508.
[I 2024-04-01 16:29:11,608] A new study created in memory with name: no-name-25df5d0c-ee3d-4afe-8198-2b380b793096
[I 2024-04-01 16:29:12,064] Trial 0 finished with value: 2247.858674180159 and parameters: {'kernel': 'rbf', 'C': 6, 'degree': 20, 'gamma': 'auto'}. Best is trial 0 with value: 2247.858674180159.
[I 2024-04-01 16:29:12,097] A new study created in memory with name: no-name-39ab6bfa-911d-45f9-ae14-5cef6fa82dac
[I 2024-04-01 16:29:12,627] Trial 0 finished with value: 570.3283690869977 and parameters: {'kernel': 'linear', 'C': 67, 'degree': 5, 'gamma': 'scale'}. Best is trial 0 with value: 570.3283690869977.
[I 2024-04-01 16:29:12,649] A new study created in memory with

,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.4,668.090478,0.911976,499.490221,0.065473,571.339337,0.934991,368.393236,0.051463
1,0.6,2201.783231,0.043948,1962.202945,0.292593,2232.206459,0.048985,1977.318145,0.283956
2,0.8,631.650993,0.921316,464.550507,0.059886,546.986998,0.943393,364.530037,0.049578
3,1.0,1553.129012,0.524284,1305.497511,0.194871,1582.369307,0.518182,1300.275671,0.186771


## Alternativ 2: Hele treningssettet

In [ ]:
# Definerer trenings- ogtestandel
test_andel = 0.2
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)


#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                                trening_dict_res=dict_res_2_design_trening,
                                rå_data=rå_data,
                                treningssett_markert=trening_markert,
                                treningssett_umarkert=trening_umarkert,
                                treningssett=trening,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                                trening_dict_res=dict_res_2_design_trening,
                                rå_data=rå_data,
                                treningssett_markert=treningssett_markert,
                                treningssett_umarkert=treningssett_umarkert,
                                treningssett=treningssett,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.73
B         13  19.70
C         13  19.70
D         12  18.18
A2         9  13.64
A1         4   6.06


# Resultater

### Trening og valideringsresultater

In [ ]:
mw_resultater_1_hel_trening = pd.DataFrame(dict_res_1_hel_trening)
mw_resultater_2_hel_trening = pd.DataFrame(dict_res_2_hel_trening)
mw_resultater_1_design_trening  = pd.DataFrame(dict_res_1_design_trening)
mw_resultater_2_design_trening  = pd.DataFrame(dict_res_1_design_trening)

### Testresultater

In [ ]:
mw_resultater_1_hel_test = pd.DataFrame(dict_res_1_hel_test)
mw_resultater_2_hel_test  = pd.DataFrame(dict_res_2_hel_test)
mw_resultater_1_design_test  = pd.DataFrame(dict_res_1_design_test)
mw_resultater_2_design_test = pd.DataFrame(dict_res_1_design_test)

### Lagring av resultater

In [ ]:
mw_resultater_1_hel_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_trening.csv')
mw_resultater_2_hel_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_trening.csv')

mw_resultater_1_design_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_trening.csv')
mw_resultater_2_design_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_trening.csv')

mw_resultater_1_hel_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_test.csv')
mw_resultater_2_hel_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_test.csv')

mw_resultater_1_design_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_test.csv')
mw_resultater_2_design_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_test.csv')

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Hele treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, \
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_hel_test_NIR,
                             trening_dict_res=dict_res_1_hel_trening_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer trenings- ogtestandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Parametervalg for kryssvalidering
splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_hel_test_NIR,
                            trening_dict_res=dict_res_2_hel_trening_NIR,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=trening_umarkert,
                            treningssett=trening,
                            testsett=testsett,
                            splitt=splitt,
                            n_runder=n_runder,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

Velger beste modell

## Designproduksjon

## Alternativ 1: Hele treningssettet

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10


# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_design_test_NIR,
                             trening_dict_res=dict_res_1_design_trening_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 2: Hele treningssettet

In [ ]:
# Definerer trenings- ogtestandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)


#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_design_test_NIR,
                                trening_dict_res=dict_res_2_design_trening_NIR,
                                rå_data=rå_data,
                                treningssett_markert=trening_markert,
                                treningssett_umarkert=trening_umarkert,
                                treningssett=trening,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Velger beste modell

## Resultater

### Trening og valideringsresultater

In [ ]:
mw_resultater_1_hel_trening_NIR = pd.DataFrame(dict_res_1_hel_trening_NIR)
mw_resultater_2_hel_trening_NIR  = pd.DataFrame(dict_res_2_hel_trening_NIR)
mw_resultater_1_design_trening_NIR   = pd.DataFrame(dict_res_1_design_trening_NIR)
mw_resultater_2_design_trening_NIR   = pd.DataFrame(dict_res_1_design_trening_NIR)

### Testresultater

In [ ]:
mw_resultater_1_hel_test_NIR = pd.DataFrame(dict_res_1_hel_test_NIR)
mw_resultater_2_hel_test_NIR  = pd.DataFrame(dict_res_2_hel_test_NIR)
mw_resultater_1_design_test_NIR  = pd.DataFrame(dict_res_1_design_test_NIR)
mw_resultater_2_design_test_NIR = pd.DataFrame(dict_res_1_design_test_NIR)

### Lagring av resultater

In [ ]:
mw_resultater_1_hel_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_trening_NIR.csv')
mw_resultater_2_hel_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_trening_NIR.csv')

mw_resultater_1_design_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_trening_NIR.csv')
mw_resultater_2_design_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_trening_NIR.csv')

mw_resultater_1_hel_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_test_NIR.csv')
mw_resultater_2_hel_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_test_NIR.csv')

mw_resultater_1_design_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_test_NIR.csv')
mw_resultater_2_design_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_test_NIR.csv')